In [ ]:
%pip install -q torch torchvision einops accelerate diffusers transformers webdataset omegaconf h5py tqdm open_clip_torch sentence-transformers deepspeed

## 1. 設定

In [ ]:
# =============================================================================
# 推論設定 - 必要に応じて変更してください
# =============================================================================

# 推論モード: "dummy" | "real"
INFERENCE_MODE = "dummy"

# 被験者ID
SUBJECT = "subj01"

# データパス
DATA_ROOT = "/content/drive/MyDrive/algonauts_2023_challenge_data"

# 学習済みモデルのパス
MODEL_CKPT = "/content/drive/MyDrive/mindeye_checkpoints/algonauts_subj01_dummy/last.pth"

# unCLIP/SDXLモデルのパス（realモード時）
UNCLIP_CKPT = None  # 例: "/content/drive/MyDrive/unclip6_epoch0_step110000.ckpt"

# 出力設定
NUM_SAMPLES = 4  # 生成するサンプル数
OUTPUT_DIR = "/content/outputs"

# =============================================================================
DUMMY_MODE = (INFERENCE_MODE == "dummy")
print(f"Inference Mode: {INFERENCE_MODE}")
print(f"Subject: {SUBJECT}")
print(f"Model: {MODEL_CKPT}")

## 2. 環境構築

In [ ]:
# GPU確認
!nvidia-smi

In [ ]:
# 依存パッケージ
!pip install -q torch torchvision einops omegaconf h5py tqdm

if not DUMMY_MODE:
    !pip install -q open_clip_torch diffusers transformers kornia

In [ ]:
# リポジトリ準備
import os
import sys

if not os.path.exists("/content/MindEyeV2"):
    !git clone https://github.com/YOUR_USERNAME/MindEyeV2.git /content/MindEyeV2

sys.path.insert(0, "/content/MindEyeV2/mysrc")
sys.path.insert(0, "/content/MindEyeV2/src")
os.chdir("/content/MindEyeV2")

# 環境変数設定
os.environ["MINDEYE_DUMMY"] = "1" if DUMMY_MODE else "0"

print(f"Working directory: {os.getcwd()}")

In [ ]:
# Google Driveマウント
from google.colab import drive
drive.mount("/content/drive")

# 出力ディレクトリ作成
os.makedirs(OUTPUT_DIR, exist_ok=True)

## 3. モデルロード

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# モジュールインポート
from algonauts_dataset import AlgonautsDataset, get_dataloader, get_total_vertices
from models_algonauts import AlgonautsMindEye
from transfer_utils import load_checkpoint, get_state_dict_from_checkpoint
from config import DEVICE

print(f"Device: {DEVICE}")

In [ ]:
# チェックポイントからモデル設定を復元
checkpoint = load_checkpoint(MODEL_CKPT)

# モデル設定（ckptに保存されていれば使用、なければデフォルト）
hidden_dim = checkpoint.get("hidden_dim", 256)
subject = checkpoint.get("subject", SUBJECT)

print(f"Checkpoint info:")
print(f"  Epoch: {checkpoint.get('epoch', 'N/A')}")
print(f"  Hidden dim: {hidden_dim}")
print(f"  Subject: {subject}")
print(f"  Final loss: {checkpoint.get('final_loss', 'N/A')}")

In [ ]:
# モデル作成
model = AlgonautsMindEye(
    subjects=[subject],
    hidden_dim=hidden_dim,
    seq_len=1,
    n_blocks=4,
    use_prior=False,
    blurry_recon=False,
)

# 重みロード
state_dict = get_state_dict_from_checkpoint(checkpoint)
model.load_state_dict(state_dict, strict=False)
model = model.to(DEVICE).eval()

print("Model loaded!")
print(f"Total vertices: {get_total_vertices(subject)}")

## 4. 画像生成器の準備

In [ ]:
if DUMMY_MODE:
    from dummy_models import DummyDiffusionEngine
    
    diffusion_engine = DummyDiffusionEngine(output_size=512).to(DEVICE)
    print("Using DummyDiffusionEngine (outputs will be random)")
    
    def generate_image(clip_tokens):
        """ダミー画像生成"""
        with torch.no_grad():
            # ダミー潜在変数
            z = torch.randn(clip_tokens.shape[0], 4, 64, 64, device=DEVICE)
            images = diffusion_engine.decode_first_stage(z)
        return images

else:
    # 本物のunCLIPを使用
    print("Loading real diffusion model...")
    
    # SGM/unCLIPのロード（要実装）
    # この部分は元のrecon_inference.ipynbを参照して実装
    raise NotImplementedError(
        "Real inference mode requires additional setup. "
        "Please implement unCLIP loading based on src/recon_inference.ipynb"
    )

## 5. データ準備

In [ ]:
# データセット
dataset = AlgonautsDataset(
    data_root=DATA_ROOT,
    subject=subject,
    split="train",
    load_images=True,
)

print(f"Dataset size: {len(dataset)}")

In [ ]:
# サンプルインデックスをランダムに選択
np.random.seed(42)
sample_indices = np.random.choice(len(dataset), NUM_SAMPLES, replace=False)
print(f"Sample indices: {sample_indices}")

## 6. 推論実行

In [ ]:
def denormalize(tensor):
    """ImageNet正規化を元に戻す"""
    mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(tensor.device)
    std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(tensor.device)
    return tensor * std + mean

def to_pil(tensor):
    """Tensor to PIL Image"""
    if tensor.dim() == 4:
        tensor = tensor[0]
    img = tensor.cpu().permute(1, 2, 0).numpy()
    img = np.clip(img * 255, 0, 255).astype(np.uint8)
    return Image.fromarray(img)

In [ ]:
# 推論実行
results = []

model.eval()
with torch.no_grad():
    for idx in sample_indices:
        sample = dataset[idx]
        
        # fMRI入力
        fmri = sample['fmri'].unsqueeze(0).to(DEVICE)
        
        # 元画像
        original_image = sample['image'].unsqueeze(0)
        
        # fMRI → CLIP tokens
        backbone, clip_voxels, _ = model(fmri)
        
        # CLIP tokens → 画像
        recon_images = generate_image(clip_voxels)
        
        results.append({
            'index': idx,
            'original': denormalize(original_image),
            'reconstruction': recon_images,
            'clip_tokens': clip_voxels,
        })
        
        print(f"Processed sample {idx}")

print(f"\nGenerated {len(results)} reconstructions")

## 7. 結果の可視化

In [ ]:
# 結果を並べて表示
fig, axes = plt.subplots(NUM_SAMPLES, 2, figsize=(8, 4 * NUM_SAMPLES))

for i, result in enumerate(results):
    # 元画像
    ax_orig = axes[i, 0] if NUM_SAMPLES > 1 else axes[0]
    orig_img = result['original'][0].cpu().permute(1, 2, 0).numpy()
    orig_img = np.clip(orig_img, 0, 1)
    ax_orig.imshow(orig_img)
    ax_orig.set_title(f"Original (idx={result['index']})")
    ax_orig.axis('off')
    
    # 再構成画像
    ax_recon = axes[i, 1] if NUM_SAMPLES > 1 else axes[1]
    recon_img = result['reconstruction'][0].cpu().permute(1, 2, 0).numpy()
    recon_img = np.clip(recon_img, 0, 1)
    ax_recon.imshow(recon_img)
    ax_recon.set_title(f"Reconstruction {'(DUMMY)' if DUMMY_MODE else ''}")
    ax_recon.axis('off')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "reconstruction_comparison.png"), dpi=150)
plt.show()

print(f"\nSaved to: {os.path.join(OUTPUT_DIR, 'reconstruction_comparison.png')}")

In [ ]:
# 個別画像を保存
for i, result in enumerate(results):
    # 元画像
    orig_pil = to_pil(result['original'])
    orig_pil.save(os.path.join(OUTPUT_DIR, f"original_{result['index']:04d}.png"))
    
    # 再構成画像
    recon_pil = to_pil(result['reconstruction'])
    recon_pil.save(os.path.join(OUTPUT_DIR, f"recon_{result['index']:04d}.png"))

print(f"Saved {len(results)} image pairs to {OUTPUT_DIR}")

## 8. CLIP特徴の分析

In [ ]:
# CLIP特徴の統計
all_clips = torch.cat([r['clip_tokens'] for r in results], dim=0)

print(f"CLIP tokens shape: {all_clips.shape}")
print(f"Mean: {all_clips.mean().item():.4f}")
print(f"Std: {all_clips.std().item():.4f}")
print(f"Min: {all_clips.min().item():.4f}")
print(f"Max: {all_clips.max().item():.4f}")

In [ ]:
# CLIP特徴のヒストグラム
plt.figure(figsize=(10, 4))
plt.hist(all_clips.cpu().numpy().flatten(), bins=100, alpha=0.7)
plt.xlabel("Value")
plt.ylabel("Count")
plt.title("Distribution of CLIP Token Values")
plt.grid(True, alpha=0.3)
plt.show()

## 次のステップ

1. **ダミーモードで動作確認** → エラーなく画像が出力されればOK
2. **realモードで本格推論** → unCLIPモデルをロードして実行
3. **定量評価** → 元画像との類似度を計算
4. **transformer_brain_encoder統合** → 画像→fMRI→画像の往復テスト

In [ ]:
!git clone https://github.com/MedARC-AI/MindEyeV2.git
%cd MindEyeV2/src

In [ ]:
from huggingface_hub import hf_hub_download
cache_dir = './'
# 軽量モデル (hidden_dim=1024, low-level module なし)
hf_hub_download('pscotti/mindeyev2', 'train_logs/multisubject_subj01_1024hid_nolow_300ep/last.pth', repo_type='dataset', local_dir=cache_dir)
hf_hub_download('pscotti/mindeyev2', 'unclip6_epoch0_step110000.ckpt', repo_type='dataset', local_dir=cache_dir)
hf_hub_download('pscotti/mindeyev2', 'bigG_to_L_epoch8.pth', repo_type='dataset', local_dir=cache_dir)
print('downloaded model files')

## fMRI ベクトルのアップロード
前処理ノートで作成した `voxel.npy` をアップロードしてください。

In [ ]:
from google.colab import files
import numpy as np, torch
uploaded = files.upload()
voxel = torch.tensor(np.load(list(uploaded.keys())[0])).float()[None,None]
print('voxel shape:', voxel.shape)

In [ ]:
import torch.nn as nn
from torchvision import transforms
from accelerate import Accelerator
import utils
from models import BrainNetwork, PriorNetwork, BrainDiffusionPrior

accelerator = Accelerator(split_batches=False, mixed_precision='fp16')
device = accelerator.device
hidden_dim = 1024  # 軽量モデル用
clip_seq_dim, clip_emb_dim = 256, 1664

# 事前に ckpt を読み込んでボクセル次元を取得
ckpt = torch.load('./train_logs/multisubject_subj01_1024hid_nolow_300ep/last.pth', map_location='cpu', weights_only=False)
expected_voxels = ckpt['model_state_dict']['ridge.linears.0.weight'].shape[1]
print('expected voxels from ckpt:', expected_voxels)

class MindEyeModule(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, x): return x

class RidgeRegression(nn.Module):
    def __init__(self, input_sizes, out_features):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(s, out_features) for s in input_sizes])
    def forward(self, x, subj_idx):
        return self.linears[subj_idx](x[:,0]).unsqueeze(1)

model = MindEyeModule()
model.ridge = RidgeRegression([expected_voxels], out_features=hidden_dim)
model.backbone = BrainNetwork(h=hidden_dim, in_dim=hidden_dim, seq_len=1, clip_size=clip_emb_dim, out_dim=clip_emb_dim*clip_seq_dim, blurry_recon=False)  # low-level なし
prior_network = PriorNetwork(dim=clip_emb_dim, depth=6, dim_head=52, heads=clip_emb_dim//52, causal=False, num_tokens=clip_seq_dim, learned_query_mode='pos_emb')
model.diffusion_prior = BrainDiffusionPrior(net=prior_network, image_embed_dim=clip_emb_dim, condition_on_text_encodings=False, timesteps=100, cond_drop_prob=0.2, image_embed_scale=None)
model.load_state_dict(ckpt['model_state_dict'], strict=False)
model.to(device).eval().requires_grad_(False)
print('model ready')

In [ ]:
import sys, os
sys.path.append(os.path.abspath('generative_models'))
from omegaconf import OmegaConf
from generative_models.sgm.models.diffusion import DiffusionEngine
config = OmegaConf.load('generative_models/configs/unclip6.yaml')
params = OmegaConf.to_container(config, resolve=True)['model']['params']
params['first_stage_config']['target'] = 'sgm.models.autoencoder.AutoencoderKL'
params['sampler_config']['params']['num_steps'] = 38
diffusion_engine = DiffusionEngine(network_config=params['network_config'], denoiser_config=params['denoiser_config'], first_stage_config=params['first_stage_config'], conditioner_config=params['conditioner_config'], sampler_config=params['sampler_config'], scale_factor=params['scale_factor'], disable_first_stage_autocast=params['disable_first_stage_autocast']).to(device)
diffusion_engine.load_state_dict(torch.load('unclip6_epoch0_step110000.ckpt', map_location='cpu')['state_dict'])
diffusion_engine.eval().requires_grad_(False)
batch = {'jpg': torch.randn(1,3,1,1).to(device), 'original_size_as_tuple': torch.ones(1,2).to(device)*768, 'crop_coords_top_left': torch.zeros(1,2).to(device)}
vector_suffix = diffusion_engine.conditioner(batch)['vector'].to(device)
print('unclip ready', vector_suffix.shape)

In [ ]:
from transformers import AutoProcessor
from modeling_git import GitForCausalLMClipEmb
processor = AutoProcessor.from_pretrained('microsoft/git-large-coco')
clip_text_model = GitForCausalLMClipEmb.from_pretrained('microsoft/git-large-coco').to(device)
clip_text_model.eval().requires_grad_(False)
class CLIPConverter(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(clip_seq_dim, 257)
        self.linear2 = nn.Linear(clip_emb_dim, 1024)
    def forward(self, x):
        x = self.linear1(x.permute(0,2,1))
        return self.linear2(x.permute(0,2,1))
clip_convert = CLIPConverter().to(device)
clip_convert.load_state_dict(torch.load('bigG_to_L_epoch8.pth', map_location='cpu')['model_state_dict'])
print('caption model ready')

In [ ]:
import matplotlib.pyplot as plt
from torchvision import transforms

# voxel 次元を ckpt に合わせる（不足はゼロ埋め、超過は切り詰め）
if voxel.shape[-1] < expected_voxels:
    pad = expected_voxels - voxel.shape[-1]
    voxel = torch.cat([voxel, torch.zeros(*voxel.shape[:-1], pad)], dim=-1)
elif voxel.shape[-1] > expected_voxels:
    voxel = voxel[..., :expected_voxels]

model.eval(); clip_text_model.eval(); diffusion_engine.eval()
with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.float16):
    voxel_d = voxel.to(device)
    voxel_ridge = model.ridge(voxel_d, 0)
    backbone, clip_voxels, _ = model.backbone(voxel_ridge)
    prior_out = model.diffusion_prior.p_sample_loop(backbone.shape, text_cond={'text_embed': backbone}, cond_scale=1., timesteps=20)
    pred_caption_emb = clip_convert(prior_out)
    generated_ids = clip_text_model.generate(pixel_values=pred_caption_emb, max_length=20)
    caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    samples = utils.unclip_recon(prior_out, diffusion_engine, vector_suffix, num_samples=1)[0].cpu()
print('caption:', caption)
plt.imshow(transforms.ToPILImage()(samples)); plt.axis('off'); plt.show()